Learning to Learn - John Tomlinson, Ahan Maurya, Dr. Jing Wang

First we need to import the modules we created for this notebook

In [3]:
import meta_network
import original_network
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Reload to pick up on-disk changes to original_network
import importlib
importlib.reload(original_network)

<module 'original_network' from 'c:\\Users\\bobtt\\OneDrive\\Desktop\\Documents\\JackCS\\learning-to-learn\\cartpole-implementation\\original_network.py'>

We will start by initializing both of our networks

In [7]:
cartpole_network, cartpole_target_network,memory, optimizer, device, env = original_network.build_model()

IndentationError: unexpected indent (2862934344.py, line 2)

Now lets build a function to graph our reward over time

In [ ]:
def plot_rewards(reward_list):
    plt.plot(reward_list)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.title('Reward over Time')
    plt.show()

Lets make a training loop to get our baseline performance for our model wihtout the meta learning

In [ ]:
NUM_EPISODES = 2000
TARGET_UPDATE = 10

In [ ]:
def training_loop(episodes):
    total_reward_list = []
    epsilon = 0.1  # Exploration rate
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = original_network.select_action(state, cartpole_network, device, epsilon, env)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            memory.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            print(action)

        original_network.optimize_model(cartpole_network, cartpole_target_network, memory, optimizer, device)

        if episode % TARGET_UPDATE == 0:
            cartpole_target_network.load_state_dict(cartpole_network.state_dict())
        
        if episode % 30 == 0:
            print(f"Episode {episode+1}/{episodes}, Total Reward: {total_reward}")
        total_reward_list.append(total_reward)
    return total_reward_list

In [2]:
plot_rewards(training_loop(NUM_EPISODES))

NameError: name 'plot_rewards' is not defined

Now that we have a baseline for our reinforcenet learning loop we can try training with our met network that will automatically shape our network.

In [ ]:
inputs_dim = env.action_space.n + 2
meta_network_test, meta_memory = meta_network.build_model(inputs_dim)

In [ ]:
def training_loop_with_meta(episodes):
    total_reward_list = []
    epsilon = 0.1  # Exploration rate
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = original_network.select_action(state, cartpole_network, device, epsilon, env)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            memory.push(state, action, (reward + shaped_reward), next_state, done)
            state = next_state
            total_reward += reward

            shaped_reward = meta_network.select_action(state, action, reward, meta_network_test, device ,epsilon, inputs_dim)
            meta_memory.push(state, action, reward, shaped_reward, next_state, done)





        original_network.optimize_model(cartpole_network, cartpole_target_network, memory, optimizer, device)

        if episode % TARGET_UPDATE == 0:
            cartpole_target_network.load_state_dict(cartpole_network.state_dict())
        
        if episode % 30 == 0:
            print(f"Episode {episode+1}/{episodes}, Total Reward: {total_reward}")
        total_reward_list.append(total_reward)
    return total_reward_list
    

IndentationError: expected an indented block after function definition on line 2 (2957932976.py, line 3)